In [92]:
import random
import time
from tqdm import tqdm
class Person:
    def __init__(self,Line):
        self.Line= Line
        self.idPath = 9
        self.Line.Path[-1] = self
        self.atdoor = 0
        self.type = "Person"
    def move(self):
        if self.idPath>=1:
            if self.Line.Path[self.idPath-1]==None:
                self.Line.Path[self.idPath-1] = self
                self.Line.Path[self.idPath] = None
                self.idPath-=1
                if self.idPath == 0:
                    self.atdoor = 2
        else:
            self.atdoor-=1
            if self.atdoor == 0:
                self.Line.Path[self.idPath] = None
                self.Line.PassingCounter+=1
class Bicycle:
    def __init__(self,Line):
        self.Line= Line
        self.headidPath = 8
        self.tailidPath = 9
        self.Line.Path[-2] = self
        self.Line.Path[-1] = self
        self.atdoor = 0
        self.type = "Bicycle"
    def move(self):
        if self.headidPath>=1:
            if self.Line.Path[self.headidPath-1]==None:
                self.Line.Path[self.headidPath-1] = self
                self.Line.Path[self.tailidPath] = None
                self.headidPath -= 1
                self.tailidPath -= 1
                if self.headidPath == 0:
                    self.atdoor = 2
        else:
            self.atdoor-=1
            self.Line.Path[self.tailidPath] = None
            if self.atdoor == 0:
                self.Line.Path[self.headidPath] = None
                self.Line.PassingCounter+=1
class Line:
    def __init__(self):
        self.Path = [None]*10
        self.PassingCounter = 0
    def update(self):
        for i in self.Path:
            if i == None:
                continue
            else:
                i.move()
    def generate(self):
        if self.Path[-1]==None:
            ifgenerate = random.random()
            if ifgenerate>=0.8:
                if self.Path[-2]==None:
                    ifperson = random.random()
                    if ifperson>=0.1:
                        Person(self)
                    else:
                        Bicycle(self)
                else:
                    Person(self)
    def print(self):
        str = "\r"
        for i in self.Path:
            if i==None:
                str = str+"🈳"
            elif i.type == "Person":
                str = str+"🚶‍♂️ "
            elif i.type == "Bicycle":
                str = str+"🚲"
        print("\r                                                            ",end=" ")
        print(str,end=" ")
        time.sleep(0.5)
    def forward(self):
        self.print()
        self.update()
        self.generate()

In [98]:
line = Line()

In [101]:
for i in range(60):
    line.update()
    line.generate()
    line.print()

🚶‍♂️ 🚶‍♂️ 🈳🚶‍♂️ 🈳🚶‍♂️ 🈳🈳🈳🈳                         

In [102]:
print(line.PassingCounter)

27
